# Feuille de travaux pratiques. Résolution numérique d'équations non linéaires

In [1]:
# chargement des bibliothèques
import numpy as np
from math import sin,cos
from math import sqrt,exp
from math import pi

%matplotlib inline
import matplotlib.pyplot as plt

## Exercice 1 (méthodes de dichotomie et de Newton-Raphson, d'après A. Quarteroni)

Dans cet exercice, on souhaite utiliser sur des exemples différentes méthodes d'approximation d'un zéro d'une fonction.

**1.** On considère tout d'abord la fonction
$$
f(x)=\frac{x}{2}-\sin(x)+\frac{\pi}{6}-\frac{\sqrt{3}}{2}
$$
sur l'intervalle $\left[-\frac{\pi}{2},\pi\right]$, en observant qu'elle y possède deux zéros.

**(a)** &Eacute;crire une fonction `f` prenant en entrée un réel $x$ et renvoyant la valeur de $f(x)$.

In [2]:
def f(x):
    return x/2-sin(x)+pi/6-sqrt(3)/2

**(b)** À l'aide du graphe de la fonction $f$ sur $\left[-\frac{\pi}{2},\pi\right]$, expliquer pourquoi la [méthode de dichotomie](http://fr.wikipedia.org/wiki/M%C3%A9thode_de_dichotomie) ne peut être utilisée que pour approcher l'un des deux zéros de $f$, que l'on notera $\xi$ dans la suite.

In [3]:
#Parce que la méthode de dichotomie permet d'approcher seulement un seul des zéros qui sont compris entre -pi/2 et
#pi, selon le signe des deux bornes dans laquelle on applique la méthode.

**(c)** Écrire une fonction `[zero,iter,res,inc]=dichotomie(f,a,b,tol,itermax)` mettant en &oelig;uvre la méthode de dichotomie pour l'approximation d'un zéro d'une fonction $f$ donnée, compris dans un intervalle $[a,b]$ tel que $f(a)f(b)<0$. En plus de la fonction et des bornes de l'intervalle, les autre paramètres d'entrée seront une tolérance `tol` pour le critère d'arrêt de la méthode et un nombre maximum `itermax` d'itérations à effectuer. Elle reverra en sortie l'approximation du zéro obtenue `zero`, le nombre d'itérations
nécessaire au calcul de cette approximation `iter`, la valeur `res` de la fonction $f$ en cette approximationt et un vecteur `inc` contenant la suite des valeurs absolues des différences entre deux approximations successives (dite suite des incréments). On réfléchira au choix du critère d'arrêt à employer.

In [4]:
def dichotomie(f,a,b,tol,itermax):
    if f(a)*f(b)>0:
        raise ValueError("les bornes sont de meme signe : dichotomie non possible")
    if f(a)>0:
        (a,b)=(b,a)
    inc =[0.5*(b+a)]
    x=b
    iter=0
    err = 0.5*(b-a)
    while (iter=<itermax and err>= tol):
        iter+=1
        x=(a+b)/2
        res=f(x)
        if res==0:
            return [x,iter,0,inc]
        if res>0:
            b=x
        else:
            a=x
        inc.append(b-a)
    return [x,iter,res,inc]

**(d)** Utiliser la fonction `dichotomie` pour calculer une approximation de $\xi$ avec une tolérance égale à $10^{-10}$ pour le critère d'arrêt à partir du choix d'un intervalle $[a,b]$ convenable.

In [5]:
print(dichotomie(f,-pi/2,pi,10**(-10),10000)[:3])

[2.2460055892509043, 34, -5.295652805159534e-11]


**(e)** Au moyen de la commande `semilogy`, tracer le graphe de la suite des incréments $|x^{(k+1)}-x^{(k)}|$ en fonction de $k$ avec une échelle semilogarithmique et déterminer la loi selon laquelle ces quantités tendent vers $0$ quand $k$ tend vers l'infini.

**(f)** Écrire une fonction `[zero,iter,res,inc]=newton(f,df,x0,tol,itermax)` qui met en &oelig;uvre la [méthode de Newton-Raphson](http://fr.wikipedia.org/wiki/M%C3%A9thode_de_Newton) pour l'approximation d'un zéro d'une fonction dérivable $f$ donnée. Les paramètres d'entrée `df`, `x0`, `tol` et `itermax` représenteront respectivement la fonction correspondant à la fonction dérivée $f'$, l'initialisation de la suite des approximations, la tolérance pour le critère d'arrêt de la méthode et le nombre maximum d'itérations à effectuer. En sortie, les paramètres seront identiques à ceux de la fonction `dichotomie`. La encore, on réfléchira au choix du critère d'arrêt à employer.

In [6]:
def newton(f,df,x0,tol,itermax):
    inc=[]
    for iter in range(1,itermax):
        x=x0
        res=f(x0)
        resd=df(x0)
        if resd==0:
            raise ValueError("La méthode ne converge pas : dérivée nulle")
        x0=x0-res/resd
        inc.append(abs(x-x0))
        if res==0:
            return [x0,iter,res,inc]
        if abs(res)<tol:
            break
    return [x0,iter,res,inc]


        

**(g)** Calculer des approximations des deux zéros $\xi$ et $\zeta$ de la fonction $f$ avec la méthode de Newton-Raphson, en prenant une tolérance égale à $10^{-10}$ pour le critère d'arrêt et comme initialisations le point $\pi$ pour $\xi$ et $-\frac{\pi}{2}$ pour $\zeta$. Comparer les nombres d'itérations effectuées pour obtenir une approximation de chacun des zéros. Pourquoi sont-ils très différents ? Comparer également les graphes des suites des incréments obtenus avec la commande `semilogy`.

In [7]:
def df(x):
    return (1/2)-cos(x)

print(newton(f,df,pi,10**(-10),1000)[:3])
print(newton(f,df,-pi/2,10**(-10),1000)[:3])

[2.2460055892979742, 5, 5.561107130347409e-13]
[-1.0472018146293844, 17, -3.14833714654128e-11]


**(h)** On cherche à réduire le nombre d'itérations nécessaires à l'obtention d'une approximation du zéro négatif $\zeta$ de la fonction $f$. La méthode de Newton-Raphson modifiée, basée sur la modification suivante de la relation de récurrence de la méthode de Newton-Raphson
$$
\forall k\in\mathbb{N},\ x^{(k+1)}=x^{(k)}−2\frac{f(x^{(k)})}{f'(x^{(k)})},
$$
a une convergence quadratique si $f'(\zeta)=0$. Mettre en &oelig;uvre cette méthode dans une fonction `modnewton` et voir combien d'itérations sont nécessaires pour qu'elle fournisse une approximation de $\zeta$ avec une tolérance égale à $10^{-10}$ pour le critère d'arrêt.

In [8]:
def modnewton(f,df,x0,tol,itermax):
    inc=[]
    for iter in range(1,itermax):
        x=x0
        res=f(x0)
        resd=df(x0)
        if resd==0:
            raise ValueError("La méthode ne converge pas : dérivée nulle")
        x0=x0-2*res/resd
        inc.append(abs(x-x0))
        if res==0:
            return [x0,iter,res,inc]
        if abs(res)<tol:
            break
    return [x0,iter,res,inc]

print(modnewton(f,df,pi,10**(-10),1000)[:3])
print(modnewton(f,df,-pi/2,10**(-10),1000)[:3])


[-1.0471975511705183, 6, -1.857403120197887e-13]
[-1.0471975512130902, 3, -8.91795526314354e-11]


**2.** On considère à présent la fonction $g(x)=x+e^{-20\,x^2}\cos(x)$, dont on veut approcher les zéros par la méthode de Newton-Raphson.

**(a)** &Eacute;crire une fonction `g` pour la fonction $g$ et une fonction `dg` pour sa dérivée $g'$.

In [9]:
def g(x):
    return x+exp((-20)*x*x)*cos(x)

def dg(x):
    return 1-40*x*exp((-20)*x*x)*cos(x)-exp((-20)*x*x)*sin(x)

**(b)** Utiliser la fonction `newton` pour essayer d'approcher d'un zéro de $g$ en prenant $x^{(0)}=0$ pour initialisation et une tolérance égale à $10^{-10}$ pour le critère d'arrêt.

In [10]:
print(newton(g,dg,0,10**(-10),1000)[:3])
#ne converge pas

[-0.99999805670506, 999, 0.9999999526024231]


**(c)** Tracer le graphe de $g$ sur l'intervalle $[-1,1]$ et tenter de donner une explication qualitative du fait la méthode de Newton-Raphson ne converge pas avec l'initialisation précédente.

**(d)** Appliquer cinq intérations de la méthode de dichotomie à la fonction $g$ sur l'intervalle $[-1,1]$ et utiliser le point obtenu comme initialisation de la méthode de Newton-Raphson pour la recherche d'un zéro de $g$.

In [11]:
print(newton(g,dg,dichotomie(g,-1,1,10**(-10),5)[0],10**(-10),1000)[:3])

[-0.25729815080399304, 5, 1.268984917146554e-13]


**3.** Modifier la fonction `dichotomie` pour obtenir une fonction
`regulafalsi` mettant en &oelig;uvre la [méthode de la fausse position](http://fr.wikipedia.org/wiki/M%C3%A9thode_de_la_fausse_position). De la même manière,
modifier la fonction `newton` pour obtenir une fonction `secante` mettant en &oelig;uvre la [méthode de la sécante](http://fr.wikipedia.org/wiki/M%C3%A9thode_de_la_s%C3%A9cante).

In [12]:
def regulafalsi(f,a,b,tol,itermax):
    if f(a)*f(b)>0:
        (a,b)=(b,a)
    tol=abs(tol)
    inc =[b-a]
    x=b
    iter=0
    while iter<itermax:
        iter+=1
        if f(b)==f(a):
            raise ValueError("La méthode ne converge pas : sécante nulle")
        x=b-f(b)*(b-a)/(f(b)-f(a))
        res=f(x)
        if res==0:
            return [x,iter,0,inc]
        if res>0:
            b=x
        else:
            a=x
        inc.append(b-a)
        if abs(f(x))<tol:
            break
    return [x,iter,res,inc]


def secante(f,x0,tol,itermax):
    inc=[]
    x=x0+1
    for iter in range(1,itermax):
        res=f(x0)
        resd=f(x)
        a=x0
        if resd==res:
            raise ValueError("La méthode ne converge pas : secante nulle")
        x0=x0-res*(x-x0)/(resd-res)
        x=a
        inc.append(abs(x-x0))
        if res==0:
            return [x0,iter,res,inc]
        if abs(res)<tol:
            break
    return [x0,iter,res,inc]

print(secante(f,pi,10**(-10),1000)[:3])
print(regulafalsi(f,0,pi,10**(-10),1000)[:3])
print(dichotomie(f,-pi/2,pi,10**(-10),10000)[:3])

[2.2460055892979742, 6, -3.3373304120232206e-13]
[2.24600558924839, 16, -5.578537631834024e-11]
[2.2460055892509043, 34, -5.295652805159534e-11]


## Exercice bonus (variantes de la méthode de la fausse position)

Le phénomène de rétention d'une des bornes d'encadrement observé lors de l'application de la méthode de la fausse position à la résolution numérique d'une équation non linéaire dans $\mathbb{R}$ a pour effet de diminuer sa vitesse de convergence, ce qui la rend parfois moins efficace que la méthode de dichotomie. Pour corriger ce défaut, plusieurs variantes ont été introduites. On propose dans cet exercice de les tester sur quelques exemples.

Pour décrire de manière explicite ces modifications, on adopte les notations suivantes. On suppose disposer initialement d'un intervalle $[x^{(0)},x^{(1)}]$ non vide de $\mathbb{R}$ et d'une application continue $f$ de $[x^{(0)},x^{(1)}]$ dans $\mathbb{R}$, telle que $f(x^{(0)})f(x^{(1)})<0$, ce qui assure l'existence d'un zéro $\xi$ de $f$. On pose alors $y^{(0)}=f(x^{(0)})$ et $y^{(1)}=f(x^{(1)})$. À l'étape $k$, avec $k$ un entier naturel non nul, on pose
$$
x^{(k+1)}=\frac{x^{(k-1)}y^{(k)}-x^{(k)}y^{(k-1)}}{y^{(k)}-y^{(k-1)}}\text{ et }y^{(k+1)}=f(x^{(k+1)}).
$$
Si $y^{(k+1)}y^{(k)}<0$, on passe à l'étape suivante. En revanche, si $y^{(k+1)}y^{(k)}>0$, on fait la mise à jour suivante
$$
x^{(k)}=x^{(k-1)}\text{ et }y^{(k)}=\alpha\,y^{(k-1)}
$$
avant de passer à l'étape suivante, avec
* $\alpha=\frac{1}{2}$ pour la <a href="https://doi.org/10.1007/BF01934364">méthode Illinois</a>,
* $\alpha=\frac{y^{(k)}}{y^{(k)}+y^{(k+1)}}$ pour la <a href="https://doi.org/10.1007/BF01932959">méthode Pegasus</a>,
* $\alpha=\frac{y^{(k)}-y^{(k+1)}}{y^{(k)}}$ si cette quantité est strictement positive, $\alpha=\frac{1}{2}$ sinon, pour la <a href="https://doi.org/10.1007/BF01951936">méthode d'Anderson-Björck</a>.

**1.** Sur le modèle de la fonction `regulafalsi` écrite dans l'exercice précédent, écrire des fonctions mettant en &oelig;uvre chacune des variantes données ci-dessus.

In [13]:
def regulafalsi2(f,a,b,tol,itermax,meth="Illinois"):
    if f(a)*f(b)>0:
        raise ValueError("les bornes sont de meme signe : regulafalsi non possible")
    if f(a)>0:
        (a,b)=(b,a)
    tol=abs(tol)
    inc =[b-a]
    (xk,xk_1)=(b,a)
    (yk,yk_1)=(f(b),f(a))
    iter=0
    while iter<itermax:
        iter+=1
        x=xk-yk*(xk-xk_1)/(yk-yk_1)
        y=f(x)
        if y==0:
            return [x,iter,y,inc]
        if yk*y>0:
            if meth=="Illinois":
                yk_1=yk_1/2
            elif meth=="Pegasus":
                yk_1=yk*yk_1/(yk+y)
                yk=y
                xk=x
        else:
            (yk,yk_1)=(y,yk)
            (xk,xk_1)=(x,xk)
        inc.append(xk-xk_1)
        if abs(y)<tol:
            break
    return [xk,iter,yk,inc]

**1.** Tester ces fonctions, ainsi que les fonctions `dichotomie` et `regulafalsi`, pour la détermination du zéro de la fonction $f(x)=11x^{11}-1$. On utilisera l'intervalle $\left[\frac{1}{10},1\right]$ comme encadrement initial et une tolérance égale à $10^{-12}$.

In [14]:
def h(x):
    return 11*(x**11)-1

print(dichotomie(h,0.1,1,10**(-12),1000)[:3])
print(regulafalsi(h,0.1,1,10**(-12),1000)[:3])
print(regulafalsi2(h,0.1,1,10**(-12),1000)[:3])#illinois
print(regulafalsi2(h,0.1,1,10**(-12),1000,"Pegasus")[:3])

[0.8041330975036887, 43, 3.3306690738754696e-13]
[0.8041330975035972, 103, -9.18487508272392e-13]
[0.8041330975036642, 17, -1.2212453270876722e-15]
[0.8041330975036644, 13, 4.440892098500626e-16]


**2.** Reprendre la question précédente avec la fonction $f(x)=1-\frac{1}{x^5}$. On utilisera l'intervalle $\left[\frac{1}{2},2\right]$ comme encadrement initial et une tolérance égale à $10^{-12}$.

In [18]:
def i(x):
    return 1-1/(x**5)

print(dichotomie(i,0.5,2,10**(-12),1000)[:3])
print(regulafalsi(i,0.5,2,10**(-12),1000)[:3])
print(regulafalsi2(i,0.5,2,10**(-12),1000)[:3])#illinois
print(regulafalsi2(i,0.5,2,10**(-12),1000,"Pegasus")[:3])

[0.9999999999998863, 42, -5.684341886080801e-13]
[1.0000000000001839, 361, 9.192646643896296e-13]
[1.0, 7, 0.0]
[1.0, 12, 0.0]


**3.** Reprendre la question précédente avec la fonction $f(x)=1-\frac{1}{x}$. On utilisera l'intervalle $\left[\frac{1}{2},2\right]$ comme encadrement initial et une tolérance égale à $10^{-12}$.

In [63]:
def j(x):
    return 1-1/(x**5)

print(dichotomie(j,0.5,2,10**(-12),1000)[:3])
print(regulafalsi(j,0.5,2,10**(-12),1000)[:3])
print(regulafalsi2(j,0.5,2,10**(-12),1000)[:3])#illinois
print(regulafalsi2(j,0.5,2,10**(-12),1000,"Pegasus")[:3])

[0.9999999999998863, 42, -5.684341886080801e-13]
[1.0000000000001839, 361, 9.192646643896296e-13]
[1.0, 7, 0.0]
[1.0, 12, 0.0]


## Exercice 2 (calcul de $\sqrt{2}$)
Dans cet exercice, on cherche à calculer une approximation de $\sqrt{2}$ de diverses façons.

**1.** On peut tout d'abord obtenir une valeur approchée de $\sqrt{2}$ en cherchant la racine positive de la fonction polynomiale $f(x)=x^2-2$. Pour cela, appliquer successivement à $f$ les méthodes de dichotomie et de de la fausse position sur l'intervalle $[1,2]$, de Newton-Raphson et de la sécante.

In [23]:
def k(x):
    return x*x-2

def dk(x):
    return 2*x

print(dichotomie(k,1,2,10**(-12),1000)[:3])
print(newton(k,dk,1,10**(-12),1000)[:3])
print(secante(k,1,10**(-12),1000)[:3])


[1.4142135623733338, 38, 6.754596881819452e-13]
[1.414213562373095, 6, 4.440892098500626e-16]
[1.414213562373095, 7, -8.881784197001252e-16]


**2.** On peut également se servir de méthodes de point fixe, définies à partir des applications suivantes
$$
g_1(x)=2+x-x^2,\ g_2(x)=\frac{2}{x}\text{ et }g_3(x)=\frac{x+2}{x+1},
$$
considérées sur l'intervalle $[1,2]$.

**(a)** Parmi les trois fonctions ci-dessus, lesquelles conduisent à une méthode de point fixe convergente ?

**(b)** Vérifier cette affirmation en calculant les vingt premiers termes des suites définies par les relations de récurrence
$$
x^{(0)}=\frac{1}{2}\text{ et },\forall k\in\mathbb{N},\ x^{(k+1)}=g_i(x^{(k)}),\ i\in\{1,2,3\}.
$$

In [25]:
def g1(x):
    return 2+x-x*x
def g2(x):
    return 2/x
def g3(x):
    return 1+1/(x+1)

x1=x2=x3=1
for i in range(20):
    print(x1,x2,x3)
    (x1,x2,x3)=(g1(x1),g2(x2),g3(x3))

1 1 1
2 2.0 1.5
0 1.0 1.4
2 2.0 1.4166666666666667
0 1.0 1.4137931034482758
2 2.0 1.4142857142857144
0 1.0 1.4142011834319526
2 2.0 1.4142156862745099
0 1.0 1.4142131979695431
2 2.0 1.4142136248948696
0 1.0 1.4142135516460548
2 2.0 1.4142135642135643
0 1.0 1.4142135620573204
2 2.0 1.4142135624272734
0 1.0 1.4142135623637995
2 2.0 1.41421356237469
0 1.0 1.4142135623728214
2 2.0 1.414213562373142
0 1.0 1.414213562373087
2 2.0 1.4142135623730965


## Exercice 3 (bassins de convergence de la méthode de Newton-Raphson)

On s'intéresse à la recherche des solutions complexes de l'équation $z^3=1$ par la méthode de Newton-Raphson. On considère pour cela la fonction d'une variable complexe $f(z)=z^3-1$, qui s'annule en chaque point $z$ du plan complexe tel que $z^3=1$.

**1.** &Eacute;crire deux fonctions `f` et `df` renvoyant respectivement les valeurs de $f(z)$ et de $f'(z)$ en un point quelconque $z$ de $\mathbb{C}$.

In [ ]:
def l(x):
    return l**3-1
def dl(x):
    return 3*x

**2.** Pour tout entier naturel $n$ supérieur ou égal à $2$, on définit une grille de pas $h=\frac{3}{n-1}$ couvrant le carré $[-1,5,1,5]\times[-1,5\mathrm{i},1,5\mathrm{i}]$.

&Eacute;crire un programme résolvant, pour une valeur donnée de $n$, l'équation $f(z)=0$ avec une tolérance égale à $10^{-4}$ par la méthode de Newton-Raphson utilisant successivement chaque point de la grille $z_{ij}=-1,5(1+\mathrm{i})+(i+\mathrm{i}j)h$, $0\leq i,j\leq n$ comme initialisation. Pour chaque couple $(i,j)$, stocker dans le tableau à deux dimensions `nrac` le numéro $k$ ($k=0$, $1$ ou $2$) de la racine cubique complexe de l'unité $e^{\mathrm{i}\frac{2k\pi}{3}}$ vers laquelle la méthode aura convergée à partir de $z_{ij}$ (on posera $k=3$ lorsque la méthode n'a pas convergé après $100$ itérations) et dans le tableau `niter` le nombre d'itérations nécessaires pour atteindre la convergence (en stockant le nombre maximal d'itérations autorisées en l'absence de convergence).

Pour automatiser le processus de reconnaissance de la racine approchée par la valeur `zero` renvoyée, on pourra utiliser les instructions suivantes (ci-dessous, `racines` désigne un tableau contenant les trois racines cubiques complexes de l'unité et `tol` est la tolérance du critère d'arrêt de la méthode de Newton-Raphson) :

`d=racines-[zero,zero,zero]
 m,k=min(abs(d)),argmin(abs(d))
 if (abs(m)>tol):
     k=3`
     
Lancer le programme avec $n$ valant $100$ et une tolérance fixée à $10^{-4}$ (compte tenu du nombre important d'appels de la méthode de Newton--Raphson).

In [ ]:
n=100
h=3/(n-1)



**3.** &Agrave; l'aide des commandes `matshow(nrac.T)` et `matshow(niter.T)`, afficher une représentation des bassins de convergence de la méthode.

**4.** Refaire des tracés pour des pas de grille plus petits (c'est-à-dire de plus grandes valeurs de $n$). Que dire des &laquo; frontières &raquo; des trois bassins de convergence de la méthode ?